<a href="https://colab.research.google.com/github/aquemi/enem_analysis/blob/main/enem_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nesse ambiente iremos importar a tabela principal com os Microdados do Enem 2017. Iremos eliminar os casos que a pessoa faltou em pelo menos uma das provas para calcular as métricas. 
Para analisar as notas das disciplinas e a nota média por cidade, estado e região, iremos importar dados de latitude e longitude dos municípios do Brasil que aplicaram a prova do ENEM pelo site do IBGE.
O arquivo final será exportado em parquet para montarmos o relatório de apresentação dos dados.

Importamos algumas bibliotecas

In [5]:
import pandas as pd
import geopandas as gpd 
import fiona 
import numpy as np

In [10]:
#pip install geopandas
!pip install pykml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 123 kB/s 


Montamos o drive e importamos os dados necessários para gerar a tabela principal.


In [2]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


A coluna Código é o do município que será utilizado para dar merge com a tabela cidades.

In [11]:
# read KML simple field
from pykml import parser
import pandas as pd

with open('/content/drive/MyDrive/enem_case/BR_Localidades_2010_v1.kml', 'r', encoding="utf-8") as f:
   root = parser.parse(f).getroot()
   
places = []
for place in root.Document.Folder.Placemark:
    #oords = place.Polygon.outerBoundaryIs.LinearRing.coordinates.text.strip()
    data = {item.get("name"): item.text for item in
            place.ExtendedData.SchemaData.SimpleData}
    places.append({"Lat": data.get('LAT'),
                  "Long": data.get('LONG'),
                  "Código": data.get('CD_GEOCODMU'),
                  })
df = pd.DataFrame(places)
print(df)

                    Lat               Long   Código
0     -30.2110754079217  -57.5570603248122  4301875
1     -29.7598231720144  -57.0818249090229  4322400
2     -30.3828679608813  -56.4536470403836  4315305
3     -30.8893840112129  -55.5348142679597  4317103
4     -29.1286368990609  -56.5571334970304  4310603
...                 ...                ...      ...
5560       -3.723805035  -38.5899275550432  2304400
5561  -3.88697288667811  -38.4451218386123  2304285
5562         -3.9061945  -38.3898029588509  2301000
5563  -4.17476322117715  -38.1232730421463  2302206
5564  -4.44980491741282  -37.7989754860384  2304459

[5565 rows x 3 columns]


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Lat     5565 non-null   object
 1   Long    5565 non-null   object
 2   Código  5565 non-null   object
dtypes: object(3)
memory usage: 130.6+ KB


In [3]:
cidades = "/content/drive/MyDrive/enem_case/cidades.csv"

In [14]:
cd = pd.read_csv(cidades) #to estimate the population 

In [15]:
cd.head(2)

,Município,Código,Gentílico,Prefeito_2017,Área_Territorial_2018,População_estimada_pessoas_2019,Densidade_demográfica_2010,Escolarização_6_a_14_anos_%_2010,IDHMunicipal_2010,Mortalidade_infantil_óbitos_por_mil_nascidos_vivos_2017,Receitas_realizadas_×1000_2017,Despesas_empenhadas_×1000_2017,PIB_per_capita_2017
0,Adamantina,3500105,adamantinense,MÁRCIO CARDIM,"411,987",35068,"82,15","97,6","0,79","11,63","160718,6793","137083,6699","30010,56"
1,Adolfo,3500204,adolfino,IZAEL ANTONIO FERNANDES,"211,055",3562,"16,85","98,6","0,73","19,23","27333,74731","20103,87713","27634,98"


In [16]:
pop = cd[['Código', 'População_estimada_pessoas_2019']]

In [27]:
pop = pop.rename({'Código': 'CO_MUNICIPIO_PROVA'}, axis = 1)
pop = pop.rename({'População_estimada_pessoas_2019': 'Populacao_(mil)'}, axis = 1)

In [20]:
pop['Populacao_(mil)'] = pop['Populacao_(mil)'].astype(float)/1000

In [28]:
pop.head()

,CO_MUNICIPIO_PROVA,Populacao_(mil)
0,3500105,35.068
1,3500204,3.562
2,3500303,36.305
3,3500402,8.180
4,3500501,18.705


In [24]:
df.head()

,Lat,Long,Código
0,-30.2110754079217,-57.5570603248122,4301875
1,-29.7598231720144,-57.0818249090229,4322400
2,-30.3828679608813,-56.4536470403836,4315305
3,-30.8893840112129,-55.5348142679597,4317103
4,-29.1286368990609,-56.5571334970304,4310603


In [22]:
arquivos = "/content/drive/MyDrive/enem_case/microdados_enem_2017/DADOS/MICRODADOS_ENEM_2017.csv"

In [23]:
# Criamos um pipeline de pré-processamento. A ideia é utilizar essa função para microdados de do ano de 2017
def pipeline_notas_Enem(arquivo):
    # Colunas a serem lidas no arquivo
    features = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT','TP_STATUS_REDACAO',
        'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT','NU_NOTA_REDACAO',
        'CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA', 'CO_UF_PROVA','SG_UF_PROVA']
    
    # Lemos o arquivo, retirando os registros em que um dos valores não estivesse presente.
    df = pd.read_csv(
        arquivo,
        #nrows = 5000, # 5k linhas para desenvolvimento inicial
        encoding = 'latin1',
        usecols = features, sep = ';').dropna()

    # Exclui casos em que a prova foi zerada, se 0 Faltou à prova, se 1	Presente na prova e se 2 Eliminado na prova:
    df.drop(df.loc[df['TP_PRESENCA_CN'] != 1].index, inplace=True)
    df.drop(df.loc[df['TP_PRESENCA_CH'] != 1].index, inplace=True)
    df.drop(df.loc[df['TP_PRESENCA_LC'] != 1].index, inplace=True)
    df.drop(df.loc[df['TP_PRESENCA_MT'] != 1].index, inplace=True)
    df.drop(df.loc[df['TP_STATUS_REDACAO'] != 1].index, inplace=True)

    # df['NU_NOTA_CN'].round(decimals = 2)
    # df['NU_NOTA_CH'].round(decimals = 2)
    # df['NU_NOTA_LC'].round(decimals = 2)
    # df['NU_NOTA_MT'].round(decimals = 2)
    # df['NU_NOTA_REDACAO'].round(decimals = 2)   
    
    df['REGIAO'] = (df['CO_MUNICIPIO_PROVA']/1000000).astype(int)
    df['NOTA_MEDIA'] = df[['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO']].mean(axis=1)
    #df['NOTA_MEDIA'].round(decimals = 2)

    return df

In [26]:
features_enem_2017 = pipeline_notas_Enem(arquivos)

In [29]:
features_enem_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4172936 entries, 0 to 6731277
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   CO_MUNICIPIO_PROVA  int64  
 1   NO_MUNICIPIO_PROVA  object 
 2   CO_UF_PROVA         int64  
 3   SG_UF_PROVA         object 
 4   TP_PRESENCA_CN      float64
 5   TP_PRESENCA_CH      float64
 6   TP_PRESENCA_LC      float64
 7   TP_PRESENCA_MT      float64
 8   NU_NOTA_CN          float64
 9   NU_NOTA_CH          float64
 10  NU_NOTA_LC          float64
 11  NU_NOTA_MT          float64
 12  TP_STATUS_REDACAO   float64
 13  NU_NOTA_REDACAO     float64
 14  REGIAO              int64  
 15  NOTA_MEDIA          float64
dtypes: float64(11), int64(3), object(2)
memory usage: 541.2+ MB


In [30]:
features_enem_2017 = features_enem_2017.merge(pop, on='CO_MUNICIPIO_PROVA', how='left')

In [31]:
df = df.rename({'Código': 'CO_MUNICIPIO_PROVA'}, axis = 1)

In [32]:
df['CO_MUNICIPIO_PROVA'] = df['CO_MUNICIPIO_PROVA'].astype(int)

In [33]:
# Unir informação da população com a tabela de microdados a partir do código do município
features_enem_2017 = features_enem_2017.merge(df, on='CO_MUNICIPIO_PROVA', how='left')

In [34]:
features_enem_2017.head(3)

,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_STATUS_REDACAO,NU_NOTA_REDACAO,REGIAO,NOTA_MEDIA,Populacao_(mil),Lat,Long
0,3503208,Araraquara,35,SP,1.0,1.0,1.0,1.0,445.4,589.0,621.5,465.5,1.0,560.0,3,536.28,236.072,-21.7903595,-48.1744399375437
1,4209300,Lages,42,SC,1.0,1.0,1.0,1.0,604.0,610.4,563.9,591.2,1.0,580.0,4,589.90,157.544,-27.8172594631137,-50.3302224241895
2,2704302,Maceió,27,AL,1.0,1.0,1.0,1.0,482.1,569.2,570.8,584.6,1.0,580.0,2,557.34,1018.948,-9.66082215163114,-35.7016299134895


In [35]:
group_uf_nota = features_enem_2017.groupby(['REGIAO','SG_UF_PROVA', 'NO_MUNICIPIO_PROVA','CO_MUNICIPIO_PROVA','Lat','Long'])\
  .agg({'NOTA_MEDIA' : 'mean',
        'NU_NOTA_CN' : 'mean',
        'NU_NOTA_CH' : 'mean',
        'NU_NOTA_LC' : 'mean',
        'NU_NOTA_MT' : 'mean',
        'NU_NOTA_REDACAO' : 'mean', 
        'Populacao_(mil)': 'mean',
        'TP_PRESENCA_CN' : 'count'})

In [38]:
group_uf_nota.head(10)

NOTA_MEDIA  \
REGIAO SG_UF_PROVA NO_MUNICIPIO_PROVA   CO_MUNICIPIO_PROVA Lat               Long                            
1      AC          Acrelândia           1200013            -10.0737938924635 -67.0523165800206  492.705882   
                   Brasiléia            1200104            -11.0164110190858 -68.7479434259584  492.840255   
                   Cruzeiro do Sul      1200203            -7.63069000500981 -72.6725313174358  488.030505   
                   Epitaciolândia       1200252            -11.0282398279564 -68.7448680666749  494.852333   
                   Feijó                1200302            -8.16024753       -70.3539836082165  493.364314   
                   Jordão               1200328            -9.19335785982848 -71.9508751248365  468.722500   
                   Marechal Thaumaturgo 1200351            -8.95359112315366 -72.7902659086983  475.036715   
                   Mâncio Lima          1200336            -7.59322259385836 -72.9165010261308  479.008793   
                   Plácido de Castro    1200385            -10.3351332827427 -67.1882600861516  483.159890   
                   Porto Acre           1200807            -9.60085823118788 -67.5405900835194  479.275467   

                                                                                                NU_NOTA_CN  \
REGIAO SG_UF_PROVA NO_MUNICIPIO_PROVA   CO_MUNICIPIO_PROVA Lat               Long                            
1      AC          Acrelândia           1200013            -10.0737938924635 -67.0523165800206  486.615468   
                   Brasiléia            1200104            -11.0164110190858 -68.7479434259584  487.255532   
                   Cruzeiro do Sul      1200203            -7.63069000500981 -72.6725313174358  479.834314   
                   Epitaciolândia       1200252            -11.0282398279564 -68.7448680666749  491.218667   
                   Feijó                1200302            -8.16024753       -70.3539836082165  480.027986   
                   Jordão               1200328            -9.19335785982848 -71.9508751248365  467.366667   
                   Marechal Thaumaturgo 1200351            -8.95359112315366 -72.7902659086983  477.944444   
                   Mâncio Lima          1200336            -7.59322259385836 -72.9165010261308  473.032455   
                   Plácido de Castro    1200385            -10.3351332827427 -67.1882600861516  473.299452   
                   Porto Acre           1200807            -9.60085823118788 -67.5405900835194  477.381867   

                                                                                                NU_NOTA_CH  \
REGIAO SG_UF_PROVA NO_MUNICIPIO_PROVA   CO_MUNICIPIO_PROVA Lat               Long                            
1      AC          Acrelândia           1200013            -10.0737938924635 -67.0523165800206  491.947495   
                   Brasiléia            1200104            -11.0164110190858 -68.7479434259584  492.796383   
                   Cruzeiro do Sul      1200203            -7.63069000500981 -72.6725313174358  480.753823   
                   Epitaciolândia       1200252            -11.0282398279564 -68.7448680666749  490.852000   
                   Feijó                1200302            -8.16024753       -70.3539836082165  485.630660   
                   Jordão               1200328            -9.19335785982848 -71.9508751248365  476.195833   
                   Marechal Thaumaturgo 1200351            -8.95359112315366 -72.7902659086983  475.097585   
                   Mâncio Lima          1200336            -7.59322259385836 -72.9165010261308  468.837032   
                   Plácido de Castro    1200385            -10.3351332827427 -67.1882600861516  480.083836   
                   Porto Acre           1200807            -9.60085823118788 -67.5405900835194  475.363200   

                                                                                                NU_NOTA_LC  \
REGIAO SG_UF_PROVA NO_M

In [37]:
group_uf_nota.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1725 entries, (1, 'AC', 'Acrelândia', 1200013, '-10.0737938924635', '-67.0523165800206') to (5, 'MT', 'Água Boa', 5100201, '-14.0490868003435', '-52.1589152986038')
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NOTA_MEDIA       1725 non-null   float64
 1   NU_NOTA_CN       1725 non-null   float64
 2   NU_NOTA_CH       1725 non-null   float64
 3   NU_NOTA_LC       1725 non-null   float64
 4   NU_NOTA_MT       1725 non-null   float64
 5   NU_NOTA_REDACAO  1725 non-null   float64
 6   Populacao_(mil)  1725 non-null   float64
 7   TP_PRESENCA_CN   1725 non-null   int64  
dtypes: float64(7), int64(1)
memory usage: 179.1+ KB


In [97]:
group_uf_nota.to_parquet('group_notas.parquet')